In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np

%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
x_init = np.random.randn(1,3).astype(np.float32)
x2_init = np.random.randn(3).astype(np.float32)
U_init = np.random.randn(3,3).astype(np.float32)
V_init = np.random.randn(3,3).astype(np.float32)
W_init = np.random.randn(3,3).astype(np.float32)
m_init = np.random.randn(1,3).astype(np.float32)

In [3]:
x = DenseTensor(x_init)
W = DenseTensor(W_init)
m = DenseTensor(m_init)
out = x.dot(W).relu()
out = out.logsoftmax()
out = out.mul(m).add(m).sum()
out.backward()

out.cpu().data, x

(array([1.0457091], dtype=float32),
 <DenseTensor <GPUBuffer with shape (1, 3)> with grad <GPUBuffer with shape (1, 3)>>)

In [4]:
x2 = DenseTensor(x2_init)#.gpu()
W = SparseTensor(W_init)
out = W.dot(x2).relu().sum()

out.backward()

out.cpu().data, x

IndexError: tuple index out of range

In [5]:
import numpy as np
import pyopencl as cl

mf = cl.mem_flags

In [6]:
dim1 = 8
dim2 = 8
dim3 = 8
bs = 4

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(4,dim3).astype(np.float32)

a.shape, b.shape

((8, 8), (4, 8))

In [7]:
def fill_sparse(mat, sparsity=0.1):
    indices = np.array(range(mat.shape[1]))
    nrows = int(mat.shape[1]*sparsity)
    for row in range(mat.shape[0]):
        lim = nrows #+ int(np.random.random()*3)
        mat[row][np.random.permutation(indices)[:lim]] = np.random.random(lim)
    return mat

a = fill_sparse(a, 0.9)
#b = fill_sparse(b, sparsity)

In [8]:
a

array([[0.59460324, 0.86305024, 0.63867457, 0.        , 0.51579547,
        0.78790121, 0.93278368, 0.02474353],
       [0.56595709, 0.53133109, 0.        , 0.54216719, 0.6889426 ,
        0.32183103, 0.64553166, 0.53694062],
       [0.80983952, 0.        , 0.73776245, 0.76804488, 0.79568409,
        0.9053589 , 0.55798573, 0.11461218],
       [0.09699463, 0.6963742 , 0.30319763, 0.51583918, 0.83718424,
        0.        , 0.05103909, 0.27944975],
       [0.75281157, 0.48989915, 0.        , 0.82777856, 0.68470816,
        0.95266816, 0.10713863, 0.63318255],
       [0.17814208, 0.84142313, 0.        , 0.9636037 , 0.76600269,
        0.79484073, 0.64250607, 0.16992201],
       [0.97628359, 0.        , 0.74801394, 0.25351422, 0.59218695,
        0.73024714, 0.80543023, 0.57306463],
       [0.89489469, 0.31771464, 0.51842438, 0.06335784, 0.80116333,
        0.60369206, 0.95093155, 0.        ]])

In [9]:
b

array([[0.3313637 , 0.5479767 , 0.32370082, 0.70595866, 0.85048527,
        0.40968382, 0.17567654, 0.5410416 ],
       [0.28101033, 0.59984696, 0.04329206, 0.92281836, 0.73979604,
        0.48108804, 0.04234902, 0.6342557 ],
       [0.5997033 , 0.73747677, 0.65285623, 0.28667638, 0.8824747 ,
        0.0911755 , 0.9040162 , 0.5719434 ],
       [0.41850978, 0.21085866, 0.11353323, 0.0624781 , 0.7506935 ,
        0.16010118, 0.3271686 , 0.6290892 ]], dtype=float32)

In [10]:
x2_init.T

array([-0.25337243, -0.7956384 ,  0.8874337 ], dtype=float32)

In [11]:
mult = a.dot(b.T)
mult.shape

(8, 4)

In [12]:
mult.shape

(8, 4)

In [13]:
def to_data(mat):
    all_rows = []
    all_idxs = []
    all_nnzs = []
    for row in range(mat.shape[0]):
        rowdata = []
        colidxs = []
        all_nnzs.append(0)
        for col in range(mat.shape[1]):
            val = mat[row][col]
            if val != 0:
                rowdata.append(val)
                colidxs.append(col)
                all_nnzs[-1] += 1
        all_rows.append(rowdata)
        all_idxs.append(colidxs)
    
    ellwidth = int(np.sqrt(np.max(all_nnzs))+1)**2
    #all_rows = np.array(all_rows)#.astype(np.float32).flatten()           
    for row in range(mat.shape[0]):
        #print(row, all_rows)
        all_rows[row] = np.array(all_rows[row])
        all_rows[row].resize(ellwidth)
        all_idxs[row] = np.array(all_idxs[row])
        all_idxs[row].resize(ellwidth)
        print(all_idxs[row])
    all_rows = np.array(all_rows).astype(np.float32).flatten()
    print(all_idxs)
    all_idxs = np.array(all_idxs).astype(np.uint32).flatten()
    all_nnzs = np.array(all_nnzs).astype(np.uint32)
    
    return all_rows, all_idxs, all_nnzs, ellwidth

In [14]:
adata, acols, annz, ellwa = to_data(a)
adata, acols, annz, ellwa

[0 1 2 4 5 6 7 0 0]
[0 1 3 4 5 6 7 0 0]
[0 2 3 4 5 6 7 0 0]
[0 1 2 3 4 6 7 0 0]
[0 1 3 4 5 6 7 0 0]
[0 1 3 4 5 6 7 0 0]
[0 2 3 4 5 6 7 0 0]
[0 1 2 3 4 5 6 0 0]
[array([0, 1, 2, 4, 5, 6, 7, 0, 0]), array([0, 1, 3, 4, 5, 6, 7, 0, 0]), array([0, 2, 3, 4, 5, 6, 7, 0, 0]), array([0, 1, 2, 3, 4, 6, 7, 0, 0]), array([0, 1, 3, 4, 5, 6, 7, 0, 0]), array([0, 1, 3, 4, 5, 6, 7, 0, 0]), array([0, 2, 3, 4, 5, 6, 7, 0, 0]), array([0, 1, 2, 3, 4, 5, 6, 0, 0])]


(array([0.59460324, 0.8630502 , 0.63867456, 0.51579547, 0.7879012 ,
        0.93278366, 0.02474353, 0.        , 0.        , 0.56595707,
        0.53133106, 0.5421672 , 0.6889426 , 0.32183105, 0.64553165,
        0.53694063, 0.        , 0.        , 0.80983955, 0.73776245,
        0.7680449 , 0.7956841 , 0.9053589 , 0.5579857 , 0.11461218,
        0.        , 0.        , 0.09699463, 0.6963742 , 0.30319762,
        0.51583916, 0.83718425, 0.05103909, 0.27944976, 0.        ,
        0.        , 0.75281155, 0.48989916, 0.8277786 , 0.6847082 ,
        0.9526682 , 0.10713863, 0.6331825 , 0.        , 0.        ,
        0.17814207, 0.84142315, 0.9636037 , 0.7660027 , 0.79484075,
        0.64250606, 0.16992201, 0.        , 0.        , 0.9762836 ,
        0.748014  , 0.25351423, 0.5921869 , 0.73024714, 0.80543023,
        0.5730646 , 0.        , 0.        , 0.8948947 , 0.31771463,
        0.5184244 , 0.06335784, 0.8011633 , 0.60369205, 0.95093155,
        0.        , 0.        ], dtype=float32),

In [15]:
a.shape

(8, 8)

In [16]:
adata.shape, acols.shape, annz.shape, ellwa

((72,), (72,), (8,), 9)

In [17]:
#acols = acols.astype(np.uint32)
#annz = annz.astype(np.uint32)

In [18]:
adata, acols, annz, b

(array([0.59460324, 0.8630502 , 0.63867456, 0.51579547, 0.7879012 ,
        0.93278366, 0.02474353, 0.        , 0.        , 0.56595707,
        0.53133106, 0.5421672 , 0.6889426 , 0.32183105, 0.64553165,
        0.53694063, 0.        , 0.        , 0.80983955, 0.73776245,
        0.7680449 , 0.7956841 , 0.9053589 , 0.5579857 , 0.11461218,
        0.        , 0.        , 0.09699463, 0.6963742 , 0.30319762,
        0.51583916, 0.83718425, 0.05103909, 0.27944976, 0.        ,
        0.        , 0.75281155, 0.48989916, 0.8277786 , 0.6847082 ,
        0.9526682 , 0.10713863, 0.6331825 , 0.        , 0.        ,
        0.17814207, 0.84142315, 0.9636037 , 0.7660027 , 0.79484075,
        0.64250606, 0.16992201, 0.        , 0.        , 0.9762836 ,
        0.748014  , 0.25351423, 0.5921869 , 0.73024714, 0.80543023,
        0.5730646 , 0.        , 0.        , 0.8948947 , 0.31771463,
        0.5184244 , 0.06335784, 0.8011633 , 0.60369205, 0.95093155,
        0.        , 0.        ], dtype=float32),

In [19]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b.T)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void matmul(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      uint gid2 = get_global_id(1);

      uint nnz    = rowNnz[gid];
      uint baseidx = gid2*nrows;
      float sum = 0;
      for (uint i = 0; i < nnz; i++) {
        uint index   = (gid * ellwidth) + i;
        uint col     = colIdx[index];
        float aval  = matData[index];
        float xval  = vector_x[baseidx+col];
        //printf("aval, xval: %.2f,%.2f: (%i,%i) \\n", aval, xval, col, index);
        sum  += aval * xval;
      }
      printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
      vector_y[baseidx+gid] = sum;
    }""").build()

In [20]:
a.shape, b.shape

((8, 8), (4, 8))

In [21]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [22]:
rows = a.shape[0]

In [23]:
mult = mult.astype(np.float32)

In [24]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.matmul  # Use this Kernel object for repeated calls
knl(queue, [rows,bs], None, adata_buf, acols_buf, annzs_buf, np.uint32(ellwa), b_buf, res_buf)

res_np = np.empty_like(b.T)
cl.enqueue_copy(queue, res_np, res_buf)

SUM/NNZ: 2.79 7 
SUM/NNZ: 2.41 7 
SUM/NNZ: 2.54 7 
SUM/NNZ: 1.86 7 
SUM/NNZ: 2.20 7 
SUM/NNZ: 2.43 7 
SUM/NNZ: 2.79 7 
SUM/NNZ: 2.75 7 
SUM/NNZ: 1.34 7 
SUM/NNZ: 1.50 7 
SUM/NNZ: 1.47 7 
SUM/NNZ: 1.08 7 
SUM/NNZ: 1.57 7 
SUM/NNZ: 1.33 7 
SUM/NNZ: 1.69 7 
SUM/NNZ: 1.51 7 
SUM/NNZ: 1.82 7 
SUM/NNZ: 1.98 7 
SUM/NNZ: 2.26 7 
SUM/NNZ: 1.75 7 
SUM/NNZ: 2.44 7 
SUM/NNZ: 2.38 7 
SUM/NNZ: 2.00 7 
SUM/NNZ: 1.78 7 
SUM/NNZ: 1.53 7 
SUM/NNZ: 2.01 7 
SUM/NNZ: 2.09 7 
SUM/NNZ: 1.73 7 
SUM/NNZ: 2.64 7 
SUM/NNZ: 2.53 7 
SUM/NNZ: 1.73 7 
SUM/NNZ: 1.45 7 


In [25]:
res_buf

In [26]:
res_np

array([[1.8154231, 1.5282668, 2.7944448, 1.3374313],
       [1.9831388, 2.0104804, 2.4146652, 1.5004594],
       [2.2570374, 2.0888033, 2.5421941, 1.467593 ],
       [1.7482169, 1.7328761, 1.8623147, 1.0750473],
       [2.436312 , 2.6403   , 2.2001545, 1.5699865],
       [2.3822901, 2.5280738, 2.4300745, 1.3315717],
       [1.998972 , 1.7276658, 2.7915556, 1.6948323],
       [1.7789352, 1.4463638, 2.749304 , 1.5135268]], dtype=float32)

In [27]:
mult

array([[1.8154231, 1.5282668, 2.7944448, 1.3374313],
       [1.9831388, 2.0104804, 2.4146655, 1.5004594],
       [2.2570372, 2.0888033, 2.5421941, 1.4675928],
       [1.748217 , 1.732876 , 1.8623147, 1.0750474],
       [2.436312 , 2.6403   , 2.2001548, 1.5699865],
       [2.38229  , 2.5280738, 2.4300745, 1.3315718],
       [1.9989719, 1.7276659, 2.791556 , 1.6948323],
       [1.7789351, 1.4463638, 2.749304 , 1.5135268]], dtype=float32)

In [28]:
res_np.shape

(8, 4)

In [29]:
mult.shape

(8, 4)

In [30]:
(res_np-mult).sum()

-2.3841858e-07

## Weight update kernel

In [31]:
bs = 4

In [32]:
dim = 8
topk = 2

x = np.random.rand(bs,dim).astype(np.float32)
y = np.random.rand(bs,dim).astype(np.float32)
x.shape,y.shape, topk

((4, 8), (4, 8), 2)

In [33]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);

      uint idx = n*gid2+gid;

      float valx = x[idx];
      float valy = y[idx];
      uint posx = 0;
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        float tempval2 = y[idx2];
        bool larger = tempval > valx;
        bool larger2 = tempval2 > valy;

        barrier(CLK_GLOBAL_MEM_FENCE);
        posx += (larger)?1:0;
        posy += (larger2)?1:0;
        barrier(CLK_GLOBAL_MEM_FENCE);
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      //printf("posx:%i", posx);
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = gid;
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
          //printf("\\nJ:%i  gid:%i", j, gid);
          //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
          barrier(CLK_GLOBAL_MEM_FENCE);
          xout[gid2*topk*topk+j*topk+gid] = res;
          barrier(CLK_GLOBAL_MEM_FENCE);
          
        }
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
    }""").build()

In [34]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             uint bs,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      //uint bs = get_global_size(1);
      //uint gid2 = get_global_id(1);

      for (uint gid2=0; gid2<bs; gid2++){
        uint idx = n*gid2+gid;

        float valx = x[idx];
        float valy = y[idx];
        uint posx = 0;
        uint posy = 0;
        for (uint i = 0; i < n; i++) {
          uint idx2 = n*gid2+i;
          float tempval = x[idx2];
          float tempval2 = y[idx2];
          bool larger = tempval > valx;
          bool larger2 = tempval2 > valy;

          posx += (larger)?1:0;
          posy += (larger2)?1:0;
        }
        //printf("posx:%i", posx);
        if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
        }
        if (posy < topk) {
          youtidx[posy+topk*gid2] = gid;
        }
      }
      for (uint gid2=0; gid2<bs; gid2++){
        if (gid < topk) {
          for (uint j=0; j<topk; j++) {
            float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
            //printf("\\nJ:%i  gid:%i", j, gid);
            //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
            //barrier(CLK_GLOBAL_MEM_FENCE);
            xout[gid2*topk*topk+j*topk+gid] = res;
          }
        }
      }
    }""").build()

In [35]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
evt = knl(queue, [dim], None, x_buf, y_buf, x_cp_buf, np.uint32(topk), np.uint32(bs), x_idx_buf, y_idx_buf)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [36]:
x

array([[0.48934165, 0.71112716, 0.29748935, 0.782211  , 0.39926562,
        0.8514658 , 0.2073185 , 0.52108914],
       [0.9927594 , 0.22187403, 0.3556742 , 0.21139157, 0.73280114,
        0.6351652 , 0.12583221, 0.74821895],
       [0.1520464 , 0.4024638 , 0.17809196, 0.4235243 , 0.36142918,
        0.41935408, 0.10778701, 0.6391944 ],
       [0.6635596 , 0.01897575, 0.52861106, 0.47245625, 0.67950535,
        0.50114566, 0.81095266, 0.5377907 ]], dtype=float32)

In [37]:
y

array([[0.13566868, 0.12836814, 0.82512575, 0.19204263, 0.21516852,
        0.6004559 , 0.34215394, 0.85616165],
       [0.51213527, 0.02328744, 0.24240972, 0.83984864, 0.3979477 ,
        0.37296107, 0.11364152, 0.265691  ],
       [0.5349935 , 0.83051544, 0.21390152, 0.34057328, 0.6541128 ,
        0.11135104, 0.32611912, 0.9343506 ],
       [0.7771125 , 0.07796126, 0.36196333, 0.9460012 , 0.9146572 ,
        0.9022052 , 0.85295373, 0.03266399]], dtype=float32)

In [38]:
x.shape, y.shape

((4, 8), (4, 8))

In [39]:
resx

array([0.7289924 , 0.6696991 , 0.7025664 , 0.64542246, 0.83376765,
       0.62839067, 0.5084271 , 0.38318932, 0.5972317 , 0.39572018,
       0.53086084, 0.35174346, 0.76716214, 0.64281285, 0.7417437 ,
       0.62151444], dtype=float32)

In [40]:
resx.reshape(bs,topk,topk)

array([[[0.7289924 , 0.6696991 ],
        [0.7025664 , 0.64542246]],

       [[0.83376765, 0.62839067],
        [0.5084271 , 0.38318932]],

       [[0.5972317 , 0.39572018],
        [0.53086084, 0.35174346]],

       [[0.76716214, 0.64281285],
        [0.7417437 , 0.62151444]]], dtype=float32)

In [41]:
resxidx

array([5, 3, 0, 7, 7, 3, 6, 4], dtype=uint32)

In [94]:
resyidx

array([7, 2, 3, 0, 7, 1, 3, 4], dtype=uint32)

In [42]:
resyidx// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint row = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint col = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }"""

array([7, 2, 3, 0, 7, 1, 3, 4], dtype=uint32)

In [43]:
idx = 0
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(8, 8)

In [44]:
xy0[382][430]

IndexError: index 382 is out of bounds for axis 0 with size 8

In [ ]:
asdf

### update vals

In [95]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)

In [96]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint col = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint row = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [97]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,bs], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa), x_cp_buf, x_idx_buf, y_idx_buf)

resa = np.empty_like(adata)
resaidx = np.zeros(acols.shape).astype(np.uint32)
resannz = np.zeros(annz.shape).astype(np.uint32)

cl.enqueue_copy(queue, resa, adata_buf)
cl.enqueue_copy(queue, resaidx, acols_buf)
cl.enqueue_copy(queue, resannz, annzs_buf)


UPDATE[6,3]: 0.767162
UPDATE[7,1]: 0.530861
UPDATE[0,0]: 0.508427
INSERT[5,2]: 0.70
UPDATE[6,4]: 0.741744
INSERT[0,3]: 0.83
UPDATE[3,1]: 0.351743
UPDATE[3,2]: 0.645422
UPDATE[4,3]: 0.642813
UPDATE[3,7]: 0.395720
UPDATE[7,0]: 0.383189
UPDATE[3,7]: 0.669699
UPDATE[4,4]: 0.621514
UPDATE[7,3]: 0.628391

In [98]:
ellwa

9

In [99]:
adata2 = adata.reshape(-1, ellwa)
adata2

array([[0.59460324, 0.8630502 , 0.63867456, 0.51579547, 0.7879012 ,
        0.93278366, 0.02474353, 0.        , 0.        ],
       [0.56595707, 0.53133106, 0.5421672 , 0.6889426 , 0.32183105,
        0.64553165, 0.53694063, 0.        , 0.        ],
       [0.80983955, 0.73776245, 0.7680449 , 0.7956841 , 0.9053589 ,
        0.5579857 , 0.11461218, 0.        , 0.        ],
       [0.09699463, 0.6963742 , 0.30319762, 0.51583916, 0.83718425,
        0.05103909, 0.27944976, 0.        , 0.        ],
       [0.75281155, 0.48989916, 0.8277786 , 0.6847082 , 0.9526682 ,
        0.10713863, 0.6331825 , 0.        , 0.        ],
       [0.17814207, 0.84142315, 0.9636037 , 0.7660027 , 0.79484075,
        0.64250606, 0.16992201, 0.        , 0.        ],
       [0.9762836 , 0.748014  , 0.25351423, 0.5921869 , 0.73024714,
        0.80543023, 0.5730646 , 0.        , 0.        ],
       [0.8948947 , 0.31771463, 0.5184244 , 0.06335784, 0.8011633 ,
        0.60369205, 0.95093155, 0.        , 0.        ]],

In [100]:
resa = resa.reshape(-1, ellwa)
resa

array([[ 0.08617616,  0.8630502 ,  0.63867456, -0.83376765,  0.51579547,
         0.7879012 ,  0.93278366,  0.02474353,  0.        ],
       [ 0.56595707,  0.53133106,  0.5421672 ,  0.6889426 ,  0.32183105,
         0.64553165,  0.53694063,  0.        ,  0.        ],
       [ 0.80983955,  0.73776245,  0.7680449 ,  0.7956841 ,  0.9053589 ,
         0.5579857 ,  0.11461218,  0.        ,  0.        ],
       [ 0.09699463,  0.34463072, -0.34222484,  0.51583916,  0.83718425,
         0.05103909, -0.7859695 ,  0.        ,  0.        ],
       [ 0.75281155,  0.48989916,  0.18496573,  0.06319374,  0.9526682 ,
         0.10713863,  0.6331825 ,  0.        ,  0.        ],
       [ 0.17814207,  0.84142315, -0.7025664 ,  0.9636037 ,  0.7660027 ,
         0.79484075,  0.64250606,  0.16992201,  0.        ],
       [ 0.9762836 ,  0.748014  , -0.5136479 , -0.14955676,  0.73024714,
         0.80543023,  0.5730646 ,  0.        ,  0.        ],
       [ 0.5117054 , -0.21314621,  0.5184244 , -0.56503284,  0

In [101]:
resa - adata2

array([[-0.5084271 ,  0.        ,  0.        , -1.3495631 , -0.27210575,
        -0.14488244,  0.9080401 ,  0.02474353,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.35174346, -0.64542246,  0.        ,  0.        ,
         0.        , -1.0654192 ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.64281285, -0.62151444,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.6661701 ,  0.19760096, -0.02883804,
         0.15233469,  0.47258407,  0.16992201,  0.        ],
       [ 0.        ,  0.        , -0.76716214, -0.7417437 ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.38318932, -0.53086084,  0.        , -0.62839067,  0

In [102]:
acols

array([0, 1, 2, 4, 5, 6, 7, 0, 0, 0, 1, 3, 4, 5, 6, 7, 0, 0, 0, 2, 3, 4,
       5, 6, 7, 0, 0, 0, 1, 2, 3, 4, 6, 7, 0, 0, 0, 1, 3, 4, 5, 6, 7, 0,
       0, 0, 1, 3, 4, 5, 6, 7, 0, 0, 0, 2, 3, 4, 5, 6, 7, 0, 0, 0, 1, 2,
       3, 4, 5, 6, 0, 0], dtype=uint32)

In [103]:
resaidx

array([0, 1, 2, 3, 4, 5, 6, 7, 0, 0, 1, 3, 4, 5, 6, 7, 0, 0, 0, 2, 3, 4,
       5, 6, 7, 0, 0, 0, 1, 2, 3, 4, 6, 7, 0, 0, 0, 1, 3, 4, 5, 6, 7, 0,
       0, 0, 1, 2, 3, 4, 5, 6, 7, 0, 0, 2, 3, 4, 5, 6, 7, 0, 0, 0, 1, 2,
       3, 4, 5, 6, 0, 0], dtype=uint32)

In [104]:
resannz

array([8, 7, 7, 7, 7, 8, 7, 7], dtype=uint32)

In [105]:
annz

array([7, 7, 7, 7, 7, 7, 7, 7], dtype=uint32)

### update vals2

In [131]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)

In [132]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint row = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint col = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [133]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,bs], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa), x_cp_buf, x_idx_buf, y_idx_buf)

resa = np.empty_like(adata)
resaidx = np.zeros(acols.shape).astype(np.uint32)
resannz = np.zeros(annz.shape).astype(np.uint32)

cl.enqueue_copy(queue, resa, adata_buf)
cl.enqueue_copy(queue, resaidx, acols_buf)
cl.enqueue_copy(queue, resannz, annzs_buf)


UPDATE[2,5]: 0.702566
UPDATE[1,7]: 0.530861
UPDATE[3,0]: 0.833768
UPDATE[0,0]: 0.508427
UPDATE[3,6]: 0.767162
UPDATE[4,6]: 0.741744
UPDATE[7,5]: 0.728992
UPDATE[1,3]: 0.351743
UPDATE[3,7]: 0.628391
UPDATE[0,7]: 0.383189
UPDATE[4,4]: 0.621514
UPDATE[2,3]: 0.645422
UPDATE[7,3]: 0.395720
UPDATE[3,4]: 0.642813
UPDATE[7,3]: 0.669699

In [134]:
ellwa

9

In [135]:
adata2 = adata.reshape(-1, ellwa)
adata2

array([[0.59460324, 0.8630502 , 0.63867456, 0.51579547, 0.7879012 ,
        0.93278366, 0.02474353, 0.        , 0.        ],
       [0.56595707, 0.53133106, 0.5421672 , 0.6889426 , 0.32183105,
        0.64553165, 0.53694063, 0.        , 0.        ],
       [0.80983955, 0.73776245, 0.7680449 , 0.7956841 , 0.9053589 ,
        0.5579857 , 0.11461218, 0.        , 0.        ],
       [0.09699463, 0.6963742 , 0.30319762, 0.51583916, 0.83718425,
        0.05103909, 0.27944976, 0.        , 0.        ],
       [0.75281155, 0.48989916, 0.8277786 , 0.6847082 , 0.9526682 ,
        0.10713863, 0.6331825 , 0.        , 0.        ],
       [0.17814207, 0.84142315, 0.9636037 , 0.7660027 , 0.79484075,
        0.64250606, 0.16992201, 0.        , 0.        ],
       [0.9762836 , 0.748014  , 0.25351423, 0.5921869 , 0.73024714,
        0.80543023, 0.5730646 , 0.        , 0.        ],
       [0.8948947 , 0.31771463, 0.5184244 , 0.06335784, 0.8011633 ,
        0.60369205, 0.95093155, 0.        , 0.        ]],

In [136]:
resa = resa.reshape(-1, ellwa)
resa

array([[ 0.08617616,  0.8630502 ,  0.63867456,  0.51579547,  0.7879012 ,
         0.93278366, -0.3584458 ,  0.        ,  0.        ],
       [ 0.56595707,  0.53133106,  0.19042373,  0.6889426 ,  0.32183105,
         0.64553165,  0.00607979,  0.        ,  0.        ],
       [ 0.80983955,  0.73776245,  0.12262243,  0.7956841 ,  0.20279253,
         0.5579857 ,  0.11461218,  0.        ,  0.        ],
       [-0.736773  ,  0.6963742 ,  0.30319762,  0.51583916,  0.1943714 ,
        -0.71612304, -0.3489409 ,  0.        ,  0.        ],
       [ 0.75281155,  0.48989916,  0.8277786 ,  0.06319374,  0.9526682 ,
        -0.63460505,  0.6331825 ,  0.        ,  0.        ],
       [ 0.17814207,  0.84142315,  0.9636037 ,  0.7660027 ,  0.79484075,
         0.64250606,  0.16992201,  0.        ,  0.        ],
       [ 0.9762836 ,  0.748014  ,  0.25351423,  0.5921869 ,  0.73024714,
         0.80543023,  0.5730646 ,  0.        ,  0.        ],
       [ 0.8948947 ,  0.31771463,  0.5184244 , -1.0020614 ,  0

In [137]:
resa - adata2

array([[-0.5084271 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.38318932,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.35174346,  0.        ,  0.        ,
         0.        , -0.53086084,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.64542246,  0.        , -0.7025664 ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.83376765,  0.        ,  0.        ,  0.        , -0.64281285,
        -0.76716214, -0.62839067,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.62151444,  0.        ,
        -0.7417437 ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -1.0654192 ,  0

In [138]:
acols

array([0, 1, 2, 4, 5, 6, 7, 0, 0, 0, 1, 3, 4, 5, 6, 7, 0, 0, 0, 2, 3, 4,
       5, 6, 7, 0, 0, 0, 1, 2, 3, 4, 6, 7, 0, 0, 0, 1, 3, 4, 5, 6, 7, 0,
       0, 0, 1, 3, 4, 5, 6, 7, 0, 0, 0, 2, 3, 4, 5, 6, 7, 0, 0, 0, 1, 2,
       3, 4, 5, 6, 0, 0], dtype=uint32)

In [139]:
resaidx

array([0, 1, 2, 4, 5, 6, 7, 0, 0, 0, 1, 3, 4, 5, 6, 7, 0, 0, 0, 2, 3, 4,
       5, 6, 7, 0, 0, 0, 1, 2, 3, 4, 6, 7, 0, 0, 0, 1, 3, 4, 5, 6, 7, 0,
       0, 0, 1, 3, 4, 5, 6, 7, 0, 0, 0, 2, 3, 4, 5, 6, 7, 0, 0, 0, 1, 2,
       3, 4, 5, 6, 0, 0], dtype=uint32)

In [140]:
resannz

array([7, 7, 7, 7, 7, 7, 7, 7], dtype=uint32)

In [141]:
annz

array([7, 7, 7, 7, 7, 7, 7, 7], dtype=uint32)

In [130]:
asdf

NameError: name 'asdf' is not defined

# OTHER

import numpy as np
import pyopencl as cl

mf = cl.mem_flags

dim = 16
topk = 4

x = np.random.rand(dim).astype(np.float32)
y = np.random.rand(dim).astype(np.float32)
x.shape,y.shape

dim1 = 4
dim2 = 8
dim3 = 1

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(dim2,dim3).flatten().astype(np.float32)

a.shape, b.shape

In [ ]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
val_out_buf = cl.Buffer(ctx, mf.READ_WRITE, 4*topk*topk)
x_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                         __global  float* y,    // INPUT
                         __global  float* xout,    // INPUT
                         uint topk,
                         __global  uint* xoutidx,    // INPUT
                         __global  uint* youtidx    // INPUT
                        ) { // LOCAL SHARED BUFFER
  uint gid = get_global_id(0);
  uint n = get_global_size(0);
  
  xout[gid] = x[gid];
  xoutidx[gid] = gid;
  youtidx[gid] = gid;
  
  float valx = x[gid];
  float valy = y[gid];
  uint posx = 0;
  uint posy = 0;
  for (uint i = 0; i < n; i++) {
    float tempval = x[i];
    float tempval2 = y[i];
    bool larger = tempval > valx;
    bool larger2 = tempval2 > valy;
      
    posx += (larger)?1:0;
    posy += (larger2)?1:0;
  }
  //printf("posx:%i", posx);
  if (posx < topk) {
    xoutidx[posx] = gid;
  }
  if (posy < topk) {
    youtidx[posy] = gid;
  }
  if (gid < topk) {
    uint i = gid;
    for (uint j=0; j<topk; j++) {
      xout[gid*topk+j] = x[xoutidx[gid]] * y[youtidx[j]];
    }
  }
}""").build()

In [ ]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
event = knl(queue, [dim,], None, x_buf, y_buf, val_out_buf, np.uint32(topk), x_idx_buf, y_idx_buf)

#event.wait()
val_out = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, val_out, val_out_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf, wait_for=[event])
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [ ]:
val_out

In [ ]:
resxidx

In [ ]:
resyidx

In [ ]:
from __future__ import division

KERNEL_CODE = """
// Thread block size
#define BLOCK_SIZE %(block_size)d
// Matrix dimensions
// (chosen as multiples of the thread block size for simplicity)
#define WA %(w_a)d // Matrix A width
#define HA %(h_a)d // Matrix A height
#define WB %(w_b)d // Matrix B width
#define HB WA  // Matrix B height
#define WC WB  // Matrix C width
#define HC HA  // Matrix C height
/*
 * Copyright 1993-2009 NVIDIA Corporation.  All rights reserved.
 *
 * NVIDIA Corporation and its licensors retain all intellectual property and
 * proprietary rights in and to this software and related documentation.
 * Any use, reproduction, disclosure, or distribution of this software
 * and related documentation without an express license agreement from
 * NVIDIA Corporation is strictly prohibited.
 *
 * Please refer to the applicable NVIDIA end user license agreement (EULA)
 * associated with this source code for terms and conditions that govern
 * your use of this NVIDIA software.
 *
 */
/* Matrix multiplication: C = A * B.
 * Device code.
 */
#define AS(j, i) As[i + j * BLOCK_SIZE]
#define BS(j, i) Bs[i + j * BLOCK_SIZE]
////////////////////////////////////////////////////////////////////////////////
//! Matrix multiplication on the device: C = A * B
//! WA is A's width and WB is B's width
////////////////////////////////////////////////////////////////////////////////
__kernel __attribute__((reqd_work_group_size(16,16,1))) 
void
matrixMul( __global float* C, __global float* A, __global float* B)
{
    __local float As[BLOCK_SIZE*BLOCK_SIZE];
    __local float Bs[BLOCK_SIZE*BLOCK_SIZE];
    // Block index
    int bx = get_group_id(0);
    int by = get_group_id(1);
    // Thread index
    int tx = get_local_id(0);
    int ty = get_local_id(1);
    // Index of the first sub-matrix of A processed by the block
    int aBegin = WA * BLOCK_SIZE * by;
    // Index of the last sub-matrix of A processed by the block
    int aEnd   = aBegin + WA - 1;
    // Step size used to iterate through the sub-matrices of A
    int aStep  = BLOCK_SIZE;
    // Index of the first sub-matrix of B processed by the block
    int bBegin = BLOCK_SIZE * bx;
    // Step size used to iterate through the sub-matrices of B
    int bStep  = BLOCK_SIZE * WB;
    // Csub is used to store the element of the block sub-matrix
    // that is computed by the thread
    float Csub = 0.0f;
    // Loop over all the sub-matrices of A and B
    // required to compute the block sub-matrix
    for (int a = aBegin, b = bBegin;
             a <= aEnd;
             a += aStep, b += bStep) {
        // Load the matrices from device memory
        // to shared memory; each thread loads
        // one element of each matrix
        AS(ty, tx) = A[a + WA * ty + tx];
        BS(ty, tx) = B[b + WB * ty + tx];
        // Synchronize to make sure the matrices are loaded
        barrier(CLK_LOCAL_MEM_FENCE);
        // Multiply the two matrices together;
        // each thread computes one element
        // of the block sub-matrix
        for (int k = 0; k < BLOCK_SIZE; ++k)
            Csub += AS(ty, k) * BS(k, tx);
        // Synchronize to make sure that the preceding
        // computation is done before loading two new
        // sub-matrices of A and B in the next iteration
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    // Write the block sub-matrix to device memory;
    // each thread writes one element
    C[get_global_id(1) * get_global_size(0) + get_global_id(0)] = Csub;
}
"""
